In [ ]:
# Libraries
import os
import pandas as pd
import psycopg2
from psycopg2 import sql
from dotenv import load_dotenv

In [ ]:
# Step 1: Load environment variables from pgadmin.env file
load_dotenv('pgadmin.env')

# Step 2: Read environment variables
db_name = os.getenv('DB_NAME')
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_host = os.getenv('DB_HOST')
db_port = os.getenv('DB_PORT')